<p align="center">Міністерство освіти і науки України НТУУ </p>
<p align="center">«Київський політехнічний інститут імені Ігоря Сікорського»</p>
<p align="center">Фізико-технічний інститут</p>


<p align="center"><strong>Спеціальні розділи програмування</strong></p>

<p align="center">Лаболаторна робота №3</p>

<p align="right">Виконав:</p>
<p align="right">Студент 2 курсу ФТІ</p>
<p align="right">Групи ФІ-31</p>

<p align="right">Горбач Ангеліна</strong></p>

In [1]:
from spyre import server
import os
import re
import urllib.request
import matplotlib.pyplot as plt

import seaborn as sns

import __main__ as main
main. __file__ ="main_file"

import pandas as pd

In [2]:
def load_vhi_data_from_directory(directory_path):
    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_data = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path, header=1, names=columns)
            df = df.drop(df[df['VHI'] == -1].index)
            province_id = filename.split('_')[2]
            df['Province_ID'] = province_id
            df = df.drop(columns=['empty']).dropna()

            df['Year'] = df['Year'].astype(str).str.extract('(\d+)', expand=False).astype(int)
            df['Week'] = df['Week'].astype(int)
            
            all_data.append(df)

    return pd.concat(all_data, ignore_index=True)

In [3]:
directory = "./Data"
vhi_data = load_vhi_data_from_directory(directory)
# print(vhi_data.tail())

In [4]:
class SimpleApp(server.App): 
    title = "NOAA Data Visualization"

    inputs = [
        {
            "type": "dropdown",
            "label": "NOAA data dropdown",
            "options": [
                {"label": "VCI", "value": "VCI"},
                {"label": "TCI", "value": "TCI"},
                {"label": "VHI", "value": "VHI"},
            ],
            "key": "ticker",
            "value": "VHI",
            "action_id": "update_data"
        },
        {
            "type": "dropdown",
            "label": "NOAA A",
            "options": [
                {"label": "1: Cherkasy", "value": 1},
                {"label": "2: Chernivtsi", "value": 2},
                {"label": "3: Chernihiv", "value": 3},
                {"label": "4: Crimea", "value": 4},
                {"label": "5: Dnipropetrovsk", "value": 5},
                {"label": "6: Donetsk", "value": 6},
                {"label": "7: Ivano-Frankivsk", "value": 7},
                {"label": "8: Kharkiv", "value": 8},
                {"label": "9: Kherson", "value": 9},
                {"label": "10: Khmelnytsky", "value": 10},
                {"label": "11: Kiev", "value": 11},
                {"label": "12: Kiev City", "value": 12},
                {"label": "13: Kirovohrad", "value": 13},
                {"label": "14: Luhansk", "value": 14},
                {"label": "15: Lviv", "value": 15},
                {"label": "16: Mykolaiv", "value": 16},
                {"label": "17: Odessa", "value": 17},
                {"label": "18: Poltava", "value": 18},
                {"label": "19: Rivne", "value": 19},
                {"label": "20: Sevastopol", "value": 20},
                {"label": "21: Sumy", "value": 21},
                {"label": "22: Ternopil", "value": 22},
                {"label": "23: Transcarpathia", "value": 23},
                {"label": "24: Vinnytsya", "value": 24},
                {"label": "25: Volyn", "value": 25},
                {"label": "26: Zaporizhzhya", "value": 26},
            ],
            "key": "area",
            "value": 15,
            "action_id": "update_data"
        },
        {
            "type": "text",
            "label": "Year Range",
            "value": "2000-2005",
            "key": "year_range",
            "action_id": "update_data"
        },
        {
            "type": "text",
            "label": "Week Interval",
            "value": "1,5",
            "key": "week",
            "action_id": "update_data"
        }
    ]
    
    controls = [{"control_type": "hidden", "label": "NOAA", "control_id": "update_data"}]
    tabs = ["Plot", "Table"]
    outputs = [
        {"type": "plot", "id": "plot", "control_id": "update_data", "tab": "Plot"},
        {"type": "table", "id": "table_id", "control_id": "update_data", "tab": "Table", "on_page_load": True}
    ]

    def getData(self, params):
        ticker = params["ticker"]
        area = int(params["area"])
        
        year_start, year_end = map(int, params["year_range"].split('-'))
        
        week_range = [int(w) for w in params["week"].split(",")]
        
        filtered_df = vhi_data[
            (vhi_data["Province_ID"] == str(area)) &
            (vhi_data["Year"] >= year_start) & (vhi_data["Year"] <= year_end) &
            (vhi_data["Week"] >= week_range[0]) & (vhi_data["Week"] <= week_range[1])
        ]
        
        return filtered_df[["Year", "Week", ticker]]

    
    def getPlot(self, params):
        data = self.getData(params)
        ticker = params["ticker"]

        sns.set_theme(style="darkgrid")
        plt.figure(figsize=(12, 6))

        unique_years = data["Year"].unique()

        for year in unique_years:
            year_data = data[data["Year"] == year]
            sns.lineplot(
                x=year_data["Week"], 
                y=year_data[ticker], 
                label=str(year)
            )

        plt.xlabel("Week")
        plt.ylabel(ticker)
        plt.legend(title="Year", loc='upper left')
        plt.grid(True)
        plt.tight_layout()

        fig = plt.gcf()
        return fig


In [ ]:
app = SimpleApp()
app.launch() 

[18/Nov/2024:11:04:56] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[18/Nov/2024:11:04:56] ENGINE Listening for SIGTERM.
[18/Nov/2024:11:04:56] ENGINE Bus STARTING
INFO:cherrypy.error:[18/Nov/2024:11:04:56] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[18/Nov/2024:11:04:56] ENGINE Set handler for console events.
INFO:cherrypy.error:[18/Nov/2024:11:04:56] ENGINE Set handler for console events.


[18/Nov/2024:11:04:56] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[18/Nov/2024:11:04:56] ENGINE Started monitor thread 'Autoreloader'.
[18/Nov/2024:11:04:56] ENGINE Serving on http://127.0.0.1:8080
INFO:cherrypy.error:[18/Nov/2024:11:04:56] ENGINE Serving on http://127.0.0.1:8080
[18/Nov/2024:11:04:56] ENGINE Bus STARTED
INFO:cherrypy.error:[18/Nov/2024:11:04:56] ENGINE Bus STARTED


127.0.0.1 - - [18/Nov/2024:11:04:57] "GET / HTTP/1.1" 200 442307 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


INFO:cherrypy.access.2245653387088:127.0.0.1 - - [18/Nov/2024:11:04:57] "GET / HTTP/1.1" 200 442307 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


127.0.0.1 - - [18/Nov/2024:11:04:58] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


...

INFO:cherrypy.access.2245653387088:127.0.0.1 - - [18/Nov/2024:11:05:08] "GET /table?ticker=VHI&area=15&year_range=2000-2005&week=1,5&output_id=table_id& HTTP/1.1" 200 6546 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


127.0.0.1 - - [18/Nov/2024:11:05:08] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


INFO:cherrypy.access.2245653387088:127.0.0.1 - - [18/Nov/2024:11:05:08] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
ERROR:root:Error: getPlot method must return an pyplot figure or matplotlib Axes object


127.0.0.1 - - [18/Nov/2024:11:05:09] "GET /plot?ticker=VHI&area=15&year_range=2000-2005&week=1,5&output_id=plot& HTTP/1.1" 200 47368 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


INFO:cherrypy.access.2245653387088:127.0.0.1 - - [18/Nov/2024:11:05:09] "GET /plot?ticker=VHI&area=15&year_range=2000-2005&week=1,5&output_id=plot& HTTP/1.1" 200 47368 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
